<div class="alert" style="background-color:#006400; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:15px 15px; color:#FFFFFF; font-size:32px'>HyperTuning for 180 Datasets</h1></div>

The work is under the **"Master Thesis"** by **Chau Tran** with the supervision from **Prof. Roland Osslen**.

## 1. Packages and Datasets

In [88]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D,AveragePooling2D,GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, Reshape, BatchNormalization, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorboard
import kerastuner as kt #(kt.tuners.RandomSearch, kt.tuners.Hyperband)
from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

from functools import partial
from matplotlib import rc, style
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
from scipy import stats

import sys, os, math, time, datetime

print("kt: ", kt.__version__)
print("tf: ", tf.__version__)
print(os.getcwd())

%matplotlib inline
%config InlineBackend.figure_format='retina'

style.use("seaborn")
pd.plotting.register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale = 1)

# rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

tf.debugging.set_log_device_placement(False)

strategy = tf.distribute.MirroredStrategy()

# Writing to file
filepath = './Version9.128timesteps/seqnetdata.ni=3.no=5.mc=15.numTimeSteps128.version9.0.csv'
with open(filepath, "r") as fp:
    [noInput, noOutput] = [int(x) for x in fp.readline().split(',')]
rdf = np.array(pd.read_csv(filepath, skiprows=1))
print(type(rdf), rdf.shape)
print(type(noInput), noInput, type(noOutput), noOutput)
display(rdf)

kt:  1.0.2
tf:  2.6.0
/home/ifeai/ChauTran/git/0_HIOF_Studying/0_MasterProject
<class 'numpy.ndarray'> (9999, 1024)
<class 'int'> 3 <class 'int'> 5


array([[0, 0, 1, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 0, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]])

## 2. Data Preparation

In [86]:
# np.set_printoptions(threshold=1000)
print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
df_test, df_train = train_test_split(rdf,test_size=0.5)
print(df_train.shape, df_test.shape)

print('Step 2: Formatting the training and testing sets and generating validating set.')
# Training and Validating sets
x_train, y_train, x_val, y_val = None, None, None, None
for i in range(df_train.shape[0]):
    df_train_i = df_train[i].reshape(-1, noInput+noOutput)
    x_train_i, y_train_i = df_train_i[:, 0:noInput], df_train_i[:, noInput:]
    x_train_i, x_val_i = train_test_split(x_train_i, test_size=0.078125)
    y_train_i, y_val_i = train_test_split(y_train_i, test_size=0.078125)
    x_train = x_train_i[np.newaxis,:,:] if x_train is None else np.append(x_train, x_train_i[np.newaxis,:,:], axis=0)
    y_train = y_train_i[np.newaxis,:,:] if y_train is None else np.append(y_train, y_train_i[np.newaxis,:,:], axis=0)
    x_val   = x_val_i[np.newaxis,:,:]   if x_val   is None else np.append(x_val, x_val_i[np.newaxis,:,:], axis=0)
    y_val   = y_val_i[np.newaxis,:,:]   if y_val   is None else np.append(y_val, y_val_i[np.newaxis,:,:], axis=0)
print("Training set: ", x_train.shape, y_train.shape)
print("Validating set: ", x_val.shape, y_val.shape)

#Testing set
x_test, y_test = None, None
for i in range(df_test.shape[0]):
    df_test_i = df_test[i].reshape(-1, noInput+noOutput)
    x_test_i, y_test_i = df_test_i[:, 0:noInput], df_test_i[:, noInput:]
    x_test = x_test_i[np.newaxis,:,:] if x_test is None else np.append(x_test, x_test_i[np.newaxis,:,:], axis=0)
print("Testing set: ", x_test.shape, y_test.shape)

Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(5000, 1024) (4999, 1024)
Step 2: Formatting the training and testing sets and generating validating set.


NameError: name 'noInput' is not defined

In [41]:
test = np.array([1, 2, 3, 4, 5, 6,1, 2, 3, 4, 5, 6]).reshape(1,-1)
display(test.shape, test)
test.reshape(-1, 3)

(1, 12)

array([[1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6]])

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])